# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=15)
lngs = np.random.uniform(low=-180.000, high=180.000, size=15)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

print("Beginning Data Retrieval")
print("-----------------------------")

response_list = []

counter = 1

for city in cities:
    
    query_url = f"{base_url}appid={weather_api_key}&units={units}&q={city}"
    
    response = requests.get(query_url).json()
    
    
    if response['cod'] == 200:
        
        print(f"Processing Record {counter} | {response['name']}")
        
        counter = counter + 1
        
        response_list.append(response) 
        
    elif response['cod'] != 200:
    
        print("City not found. Skipping...")

In [ ]:
#test code for looking at format of the json

response = requests.get(base_url + "appid=" + weather_api_key + "&q=" + "Jamestown").json()
# weather_json = weather_response.json()

print(json.dumps(response))

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# create a dictionary of empty lists
weather_data = {
    "City": [],
    "Cloudiness": [],
    "Date": [],
    "Humidity": [],
    "Latitude": [],
    "Longitude": [],
    "Max Temp (C)": [],
    "Wind Speed": []
}

# populate lists with 
for response in range(0, len(response_list)):
    weather_data["City"].append(response_list[response]['name'])
    weather_data["Cloudiness"].append(response_list[response]['clouds']['all'])
    weather_data["Date"].append(response_list[response]['dt'])
    weather_data["Humidity"].append(response_list[response]['main']['humidity'])
    weather_data["Latitude"].append(response_list[response]['coord']['lat'])
    weather_data["Longitude"].append(response_list[response]['coord']['lon'])
    weather_data["Max Temp (C)"].append(response_list[response]['main']['temp_max'])
    weather_data["Wind Speed"].append(response_list[response]['wind']['speed'])

#turn dictionary into dataframe
weather_df = pd.DataFrame(weather_data)
    
    
weather_df.head()

In [ ]:
# export to CSV
output = "../output.csv"
weather_df.to_csv(output, index=False)

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

In [ ]:
#create a function to create each scatterplot

def scatterplots(x_value, y_value, x_label, y_label):
    
    #title and labels
    date = weather_df["Date"].max()
    date = time.strftime("(%m/%d/%y)", time.localtime(date))
    title = f"{x_label} vs. {y_label} {date}"
    
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    
    #data points
    x_axis = weather_df[x_value]
    y_axis = weather_df[y_value]

    #create the scatter plot
    plt.scatter(x_axis, y_axis, c=y_axis)

    #visual improvements
    plt.grid(axis='both')
    plt.tight_layout()

    plt.show()

#### Latitude vs. Temperature Plot

In [ ]:
scatterplots("Latitude", "Max Temp (C)", "Latitude", "Max Temperature (C)")

#### Latitude vs. Humidity Plot

In [ ]:
scatterplots("Latitude", "Humidity", "Latitude", "Humidity (%)")

#### Latitude vs. Cloudiness Plot

In [ ]:
scatterplots("Latitude", "Cloudiness", "Latitude", "Cloudiness (%)")

#### Latitude vs. Wind Speed Plot

In [ ]:
scatterplots("Latitude", "Wind Speed", "Latitude", "Wind Speed (mph)")

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots
def regression(hemisphere, x_column, y_column, x_label, y_label, x, y):

    #title
    title = f"{hemisphere} - {x_label} vs. {y_column}"

    #determine which hemisphere
    if hemisphere == "Southern Hemisphere":
        data_frame = southern[[x_column, y_column]]
    
        #set x and y
        x_values = southern[x_column]
        y_values = southern[y_column]
    
    elif hemisphere == "Northern Hemisphere":
        data_frame = northern[[x_column, y_column]]
    
        #set x and y
        x_values = northern[x_column]
        y_values = northern[y_column]

    #run linear regression
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

    #calculate regression values and r squared
    regress_values = x_values * slope + intercept
    r_squared = rvalue ** 2

    #find equation for regression line
    line_eq = f"y = {round(slope, 2)}x + {round(intercept, 2)}"

    #scatterplot
    plt.scatter(x_values, y_values)

    #add regression line
    plt.plot(x_values, regress_values, "r-")

    #show equation on plot
    plt.annotate(line_eq, (x, y), fontsize=15)

    #title and labels
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)

    #print r squared value
    print(f"The r-squared value is: {r_squared}")

    plt.show()

In [ ]:
# Create Northern and Southern Hemisphere DataFrames
southern = weather_df.loc[weather_df["Latitude"] < 0]
northern = weather_df.loc[weather_df["Latitude"] > 0]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
regression("Northern Hemisphere", "Latitude", "Max Temp (C)", "Latitude", "Max Temp (C)", 5, 0)

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
regression("Southern Hemisphere", "Latitude", "Max Temp (C)", "Latitude", "Max Temp (C)", -55, 12.5)

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
regression("Northern Hemisphere", "Latitude", "Humidity", "Latitude", "Humidity (%)", 5, 65)

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
regression("Southern Hemisphere", "Latitude", "Humidity", "Latitude", "Humidity (%)", -35, 45)

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
regression("Northern Hemisphere", "Latitude", "Cloudiness", "Latitude", "Cloudiness (%)", 30, 90)

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
regression("Southern Hemisphere", "Latitude", "Cloudiness", "Latitude", "Cloudiness (%)", -35, 45)

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
regression("Northern Hemisphere", "Latitude", "Wind Speed", "Latitude", "Wind Speed (mph)", 5, 4)

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
regression("Southern Hemisphere", "Latitude", "Wind Speed", "Latitude", "Wind Speed (mph)", -35, 6)